In [1]:
import pandas as pd
import torch

from scripts.training import run_epoch, train_loop
from utils import move_data_to_device, move_dict_to_cpu, plot_image, plot_image_boxes
from PsfSimulator import PsfDataset
from models.subpix_rcnn import SubpixRCNN

import torch.nn as nn
from torchvision.ops import MultiScaleRoIAlign
from torchvision.ops import misc as misc_nn_ops
from torchvision.transforms._presets import ObjectDetection
from torchvision.models._api import register_model, Weights, WeightsEnum
from torchvision.models._meta import _COCO_CATEGORIES
from torchvision.models._utils import _ovewrite_value_param, handle_legacy_interface
from torchvision.models.resnet import resnet50, ResNet50_Weights
from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers
from torchvision.models.detection.faster_rcnn import _default_anchorgen, FasterRCNN, FastRCNNConvFCHead, RPNHead
from torchvision.models.detection.roi_heads import RoIHeads

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN
import torch.nn.functional as F
import torch.optim as optim
import os
import datetime

c:\Users\zeiss\anaconda3\envs\hilger312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Density 1 = 40 spots per 64x64 image
# Density 0.05 = 2 spots per 64x64 image

In [2]:
# Instatiate the dataset
seed = None

num_spots_min = 2
num_spots_max = 80
sigma_mean= 1.0
sigma_std = 0.1
snr_min = 2
snr_max = 20
snr_std = 0.0
base_noise_min = 50
base_noise_max = 6000
use_gauss_noise = False
gauss_noise_std = 0.02
use_perlin_noise = False
perlin_min_max = (0.4, 0.6)
img_w = 64
img_h = 64

train_dataset = PsfDataset(seed, 1000, num_spots_min, num_spots_max, sigma_mean, sigma_std,
                      snr_min, snr_max, snr_std, base_noise_min, base_noise_max, use_gauss_noise,
                      gauss_noise_std, use_perlin_noise, perlin_min_max, img_w, img_h)


valid_dataset = PsfDataset(seed, 50, num_spots_min, num_spots_max, sigma_mean, sigma_std,
                      snr_min, snr_max, snr_std, base_noise_min, base_noise_max, use_gauss_noise,
                      gauss_noise_std, use_perlin_noise, perlin_min_max, img_w, img_h)


# Print the number of samples in the training and validation datasets
print(pd.Series({
    'Training dataset size:': len(train_dataset),
    'Validation dataset size:': len(valid_dataset)}))

data_loader_params = {'batch_size':4,    'collate_fn': lambda batch: tuple(zip(*batch)),}

training_loader = torch.utils.data.DataLoader(train_dataset, **data_loader_params)
validation_loader = torch.utils.data.DataLoader(valid_dataset, **data_loader_params)

Training dataset size:      1000
Validation dataset size:      50
dtype: int64


In [3]:
def generate_checkpoint_path(model_name, project_name):

  # 1. Define the project directory within Colab's content area
  folder_path = os.path.join(os.curdir, project_name)

  # 2. Create the directory if it doesn't exist
  os.makedirs(folder_path, exist_ok=True)

  # 3. Generate a timestamped subdirectory
  timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  checkpoint_dir = os.path.join(folder_path, timestamp)
  os.makedirs(checkpoint_dir, exist_ok=True)

  # 4. Construct the checkpoint path
  checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}.pth")

  return checkpoint_path

In [4]:
from torch.utils.tensorboard import SummaryWriter

In [7]:
writer = SummaryWriter('runs/psf_rcnn2')

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backbone = resnet_fpn_backbone("resnet50", pretrained=True)
kwargs = {"nms_tresh": 0.1, "detections_per_img": None, "score_thresh": 0.8}

model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)
model.to(device)
model.name = "testmodel"
optimizer = optim.Adam(model.parameters(), lr=0.0001)

checkpoint_path = generate_checkpoint_path("second_long_run", "subpix_rcnn_models")
num_epochs = 200

train_loop(model, training_loader, validation_loader, optimizer, device, num_epochs, checkpoint_path, writer)

Custom SubpixRoIHeads successfully initialized!


Epochs:   0%|          | 0/200 [00:00<?, ?it/s]




















































































































































































































































































































































































































































































































Train: 100%|██████████| 250/250 [00:59<00:00,  4.22it/s, loss=170, avg_loss=0.68, data_time=7371.276s, train_time=28.934s]


























Epochs:   0%|          | 1/200 [01:01<3:22:25, 61.03s/it]

New best loss: 0.45962050327887904





























































































































































































































































































































































































































































































Epochs:   0%|          | 1/200 [01:57<6:31:13, 117.96s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from perlin_numpy import generate_perlin_noise_2d
